In [1]:
import torch
from vllm import LLM, SamplingParams
import pandas as pd
from PIL import Image
from pyprojroot import here
from src.utils import preprocess_messages
from src.lm import SYSTEM_PROMPT
from transformers import AutoProcessor

/scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 08-08 13:51:10 [__init__.py:235] Automatically detected platform cuda.


In [2]:
!export LD_LIBRARY_PATH="/scr/benpry/conda/envs/vtc/lib64:/scr/benpry/conda/envs/vtc/lib:$LD_LIBRARY_PATH"
!export CUDA_HOME="/usr/local/cuda-12.6"

In [ ]:
MODEL = "Qwen/Qwen2.5-VL-7B-Instruct"
IMAGE = Image.open(here("data/compiled_grid.png"))

In [ ]:
df = pd.read_csv(here("context_prep/yoked.csv"))

In [5]:
processor = AutoProcessor.from_pretrained(MODEL)
df["chat_prompt"] = df.apply(preprocess_messages, axis=1)
df["chat_messages"] = df["chat_prompt"].apply(
    lambda x: [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": SYSTEM_PROMPT},
                {"type": "image", "image": IMAGE},
            ],
        },
        *x,
    ]
)
df["formatted_prompt"] = df["chat_messages"].apply(
    lambda x: processor.apply_chat_template(
        x, tokenize=False, add_generation_prompt=True
    )
)

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
/sailhome/benpry/vlm-tg-context/src/utils.py:33: UserWarning: Message {'role': 'describer', 'message_number': 2} is missing 'text' field.
  warnings.warn(f"Message {message} is missing 'text' field.")


In [6]:
row = df.iloc[1]
print(row["chat_messages"])
print(row["formatted_prompt"])

[{'role': 'system', 'content': [{'type': 'text', 'text': 'You will be presented with a list of messages between people playing a reference game, where the describer has to get the matcher to choose an image from a list of images. Your goal is to guess which of the images the describer is trying to get the matcher to choose. The images, with their labels, are shown in the image.\nPlease answer with just the letter corresponding to the image you think the describer is trying to get the matcher to choose.\n'}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=860x790 at 0x7FB7AC144BC0>}]}, {'role': 'user', 'content': 'describer: triangle at the bottom, right angled triangle above, 2 rotated squares on top of that They are touching on their points\n'}, {'role': 'assistant', 'content': 'K'}, {'role': 'user', 'content': 'describer: This one looks like a person sat down Their head is a rotated square\nmatcher: do they have bunny ears\nmatcher: or knee up\nmatcher

In [7]:
llm = LLM(
    model=MODEL,
    dtype=torch.bfloat16,
    tensor_parallel_size=2,
    max_model_len=8192,
    max_num_seqs=8,
    max_logprobs=100,
)

INFO 08-08 13:51:36 [config.py:1604] Using max model len 8192


2025-08-08 13:51:38,191	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 08-08 13:51:38 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 08-08 13:51:38 [__init__.py:2899] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 08-08 13:51:42 [__init__.py:235] Automatically detected platform cuda.
INFO 08-08 13:51:43 [core.py:572] Waiting for init message from front-end.
INFO 08-08 13:51:43 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='Qwen/Qwen2.5-VL-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, di

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.78it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:01<00:00,  2.97it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:01<00:00,  2.77it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:01<00:00,  2.52it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:01<00:00,  2.56it/s]
(VllmWorker rank=0 pid=1248266) 


(VllmWorker rank=0 pid=1248266) INFO 08-08 13:51:54 [default_loader.py:262] Loading weights took 2.00 seconds
(VllmWorker rank=0 pid=1248266) INFO 08-08 13:51:55 [gpu_model_runner.py:1892] Model loading took 7.8681 GiB and 2.584217 seconds
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:51:55 [default_loader.py:262] Loading weights took 2.39 seconds
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:51:55 [gpu_model_runner.py:1892] Model loading took 7.8681 GiB and 3.112804 seconds
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:51:56 [gpu_model_runner.py:2380] Encoder cache will be initialized with a budget of 16384 tokens, and profiled with 1 image items of the maximum feature size.
(VllmWorker rank=0 pid=1248266) INFO 08-08 13:51:56 [gpu_model_runner.py:2380] Encoder cache will be initialized with a budget of 16384 tokens, and profiled with 1 image items of the maximum feature size.


...

(VllmWorker rank=0 pid=1248266) INFO 08-08 13:52:08 [backends.py:530] Using cache directory: /afs/cs.stanford.edu/u/benpry/.cache/vllm/torch_compile_cache/01442316c6/rank_0_0/backbone for vLLM's torch.compile
(VllmWorker rank=0 pid=1248266) INFO 08-08 13:52:08 [backends.py:541] Dynamo bytecode transform time: 6.11 s
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:52:09 [backends.py:530] Using cache directory: /afs/cs.stanford.edu/u/benpry/.cache/vllm/torch_compile_cache/01442316c6/rank_1_0/backbone for vLLM's torch.compile
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:52:09 [backends.py:541] Dynamo bytecode transform time: 7.10 s
(VllmWorker rank=0 pid=1248266) INFO 08-08 13:52:13 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 5.016 s
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:52:15 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 4.983 s
(VllmWorker rank=1 pid=1248267) INFO 08-08 13:52:17 [monit

(VllmWorker rank=1 pid=1248267) /scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
(VllmWorker rank=1 pid=1248267) If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
(VllmWorker rank=1 pid=1248267)   warnings.warn(
(VllmWorker rank=0 pid=1248266) /scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
(VllmWorker rank=0 pid=1248266) If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
(VllmWorker rank=0 pid=1248266)   warnings.warn(


(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546] WorkerProc hit an exception.
(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546] Traceback (most recent call last):
(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546]   File "/scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/flashinfer/jit/cpp_ext.py", line 199, in run_ninja
(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546]     subprocess.run(
(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546]   File "/scr/benpry/conda/envs/vtc/lib/python3.12/subprocess.py", line 571, in run
(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546]     raise CalledProcessError(retcode, process.args,
(VllmWorker rank=0 pid=1248266) ERROR 08-08 13:52:17 [multiproc_executor.py:546] subprocess.CalledProcessError: Command '['ninja', '-v', '-C', '/sailhome/benpry/.cache/flashinfer/8

Process EngineCore_0:
Traceback (most recent call last):
  File "/scr/benpry/conda/envs/vtc/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/scr/benpry/conda/envs/vtc/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 636, in run_engine_core
    raise e
  File "/scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 623, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 441, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/scr/benpry/conda/envs/vtc/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 86, in __init__
    self._initialize_kv_caches(vllm_config)
  File "/

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

In [ ]:
out = llm.generate(df.iloc[1]["formatted_prompt"])